In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/Pneumonia.zip -d /content/drive/MyDrive/Pneumonia_data/

In [4]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Image data generator for augmentation
datagen = ImageDataGenerator(
    rescale=1./255,            # Normalize the pixel values
    rotation_range=15,         # Randomly rotate images by 15 degrees
    width_shift_range=0.1,     # Randomly shift images horizontally (10%)
    height_shift_range=0.1,    # Randomly shift images vertically (10%)
    shear_range=0.2,           # Apply random shear transformations
    zoom_range=0.2,            # Randomly zoom into images
    brightness_range=[0.8, 1.2],  # Adjust brightness
    fill_mode='nearest'        # Fill missing pixels with the nearest pixel
)

# Load data from the directory
train_data = datagen.flow_from_directory(
    '/content/drive/MyDrive/Pneumonia_data/Pneumonia/chest_xray/train',
    target_size=(224, 224),  # Resize to match input size for model
    batch_size=8,            # Small batch size for efficient training
    class_mode='categorical'  # Two classes: 'normal' and 'pneumonia'
)

validation_data = datagen.flow_from_directory(
    '/content/drive/MyDrive/Pneumonia_data/Pneumonia/chest_xray/val',
    target_size=(224, 224),
    batch_size=8,
    class_mode='categorical'
)

from tensorflow.keras.applications import EfficientNetB0, DenseNet121
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, Concatenate, Flatten

# Input layer
input_shape = (224, 224, 3)
inputs = tf.keras.Input(shape=input_shape)

# EfficientNetB0 feature extraction
efficient_net = EfficientNetB0(include_top=False, weights='imagenet', input_tensor=inputs)
efficient_net_output = GlobalAveragePooling2D()(efficient_net.output)

# DenseNet121 feature extraction
dense_net = DenseNet121(include_top=False, weights='imagenet', input_tensor=inputs)
dense_net_output = GlobalAveragePooling2D()(dense_net.output)

# Concatenate features from both networks
concatenated_features = Concatenate()([efficient_net_output, dense_net_output])

# Add Dropout for regularization
concatenated_features = Dropout(0.3)(concatenated_features)

# Final classification layer
outputs = Dense(2, activation='softmax')(concatenated_features)
model = tf.keras.Model(inputs, outputs)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
29084464/29084464 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


In [5]:
# Train the model
history = model.fit(train_data, validation_data=validation_data, epochs=5)

Epoch 1/5


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


652/652 ━━━━━━━━━━━━━━━━━━━━ 500s 286ms/step - accuracy: 0.8731 - loss: 0.3173 - val_accuracy: 0.5000 - val_loss: 2.6152
Epoch 2/5
652/652 ━━━━━━━━━━━━━━━━━━━━ 170s 259ms/step - accuracy: 0.9408 - loss: 0.1611 - val_accuracy: 0.8750 - val_loss: 0.3575
Epoch 3/5
652/652 ━━━━━━━━━━━━━━━━━━━━ 165s 252ms/step - accuracy: 0.9588 - loss: 0.1172 - val_accuracy: 0.8125 - val_loss: 0.6075
Epoch 4/5
652/652 ━━━━━━━━━━━━━━━━━━━━ 202s 252ms/step - accuracy: 0.9632 - loss: 0.1041 - val_accuracy: 0.8125 - val_loss: 0.4942
Epoch 5/5
652/652 ━━━━━━━━━━━━━━━━━━━━ 168s 255ms/step - accuracy: 0.9665 - loss: 0.0933 - val_accuracy: 0.6875 - val_loss: 0.5029


In [6]:
# Image data generator for the test set (no augmentation, just rescaling)
test_datagen = ImageDataGenerator(rescale=1./255)

# Load the test data
test_data = test_datagen.flow_from_directory(
    '/content/drive/MyDrive/Pneumonia_data/Pneumonia/chest_xray/test',  # Path to the test dataset
    target_size=(224, 224),  # Resize to match input size for model
    batch_size=8,            # Batch size can be the same as training
    class_mode='categorical',  # Two classes: 'normal' and 'pneumonia'
    shuffle=False            # Important: don't shuffle for evaluation
)


# Evaluate the model
test_loss, test_acc = model.evaluate(test_data)
print(f'Test accuracy: {test_acc}')


Found 624 images belonging to 2 classes.
78/78 ━━━━━━━━━━━━━━━━━━━━ 9s 116ms/step - accuracy: 0.9313 - loss: 0.2259
Test accuracy: 0.9214743375778198


In [7]:
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

# Evaluate on the test data
test_loss, test_acc = model.evaluate(test_data)

# Predict on test data to get confusion matrix, precision, recall, etc.
y_true = test_data.classes
y_pred_probs = model.predict(test_data)
y_pred = y_pred_probs.argmax(axis=1)

# Confusion Matrix
print(confusion_matrix(y_true, y_pred))

# Classification Report (precision, recall, f1-score)
print(classification_report(y_true, y_pred, target_names=['Normal', 'Pneumonia']))

# AUC score
auc = roc_auc_score(y_true, y_pred_probs[:, 1])
print(f"AUC: {auc}")

78/78 ━━━━━━━━━━━━━━━━━━━━ 7s 92ms/step - accuracy: 0.9313 - loss: 0.2259
78/78 ━━━━━━━━━━━━━━━━━━━━ 23s 104ms/step
[[216  18]
 [ 31 359]]
              precision    recall  f1-score   support

      Normal       0.87      0.92      0.90       234
   Pneumonia       0.95      0.92      0.94       390

    accuracy                           0.92       624
   macro avg       0.91      0.92      0.92       624
weighted avg       0.92      0.92      0.92       624

AUC: 0.9765066841989919


In [8]:
model.save("model.h5")

In [9]:
model.save("model.keras")

In [10]:
tf.saved_model.save(model, "saved_model")